### Problema de Negócio

Classificar os sentimentos do cliente (POSITIVOS ou NEGATIVO) em relação aos tweets registrados por eles na rede social Twitter, que referenciam a empresa alvo.

Tentar comprrender o que os clientes, parceiros e fornecedores estão pensando sobre a empresa (produto, serviço).

Total de caracteres por tweet: 140.

Em outras palavras, será treinado um modelo com base nos dados JÁ CLASSIFICADOS e fornecidos pela Universidade de Michigan, para que, à partir deste modelo, o mesmo classifique as demais instâncias (tweets) coletadas.

Fonte: https://www.kaggle.com/c/si650winter11

DataSet Treino: https://www.kaggle.com/c/2558/download/training.txt

DataSet Teste: https://www.kaggle.com/c/2558/download/testdata.txt

Quantidade de tweets classificados:  1.578.627

Classe: Sentimento

Valores possíveis da Classe:

1 -> positivo

0 -> negativo

In [5]:
# Pacotes necessários:
#requests_oauthlib -> biblioteca para autenticação
#twython -> biblioteca que conecta ao Twitter e captura os tweet's
#nltk -> processamento de linguagem natural

In [2]:
# Módulos do Spark
from pyspark.streaming import StreamingContext
from pyspark import SparkContext

# OAuth1Session = módulo da biblioteca requests_oauthlib que, ao passat determinadas credenciais, tenta efetuar a autenticação com o método request (HTTP).
from requests_oauthlib import OAuth1Session

# add = módulo da biblioteca operator que faz uma operação intrínseca de soma entre elementos passados na função.
from operator import add
import requests_oauthlib

# Módulo para manipulação de tempo (horas, minutos, segundos, milésimos).
from time import gmtime, strftime

# requests = biblioteca de funções para chamadas do método REQUEST em aplicações HTTP.
import requests

# time = biblioteca de funções para manipulação de tempo (horas, minutos, segundos, milésimos).
import time

# string = biblioteca de funções para manipulação de strings.
import string

# ast = biblioteca de funções usada para operações gramaticais.
import ast

# json = biblioteca de funções usada para manipulação de arquivos no formato JSON.
import json

# Módulos do NLTK (Natural Language Tool Kit)
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

### 1. Definindo o tempo de atualização do Streaming Context...

In [3]:
INTERVAL = 5
SContext = StreamingContext(sc, INTERVAL)

In [14]:
file = sc.textFile('/home/daniellj/Projetos/Github/SentimentAnalysisTwitter/datasets/data_sentiment_analyses.csv')
file.take(10)

['ItemID,Sentiment,SentimentSource,SentimentText',
 '1,0,Sentiment140,                     is so sad for my APL friend.............',
 '2,0,Sentiment140,                   I missed the New Moon trailer...',
 '3,1,Sentiment140,              omg its already 7:30 :O',
 "4,0,Sentiment140,          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...",
 '5,0,Sentiment140,         i think mi bf is cheating on me!!!       T_T',
 '6,0,Sentiment140,         or i just worry too much?        ',
 '7,1,Sentiment140,       Juuuuuuuuuuuuuuuuussssst Chillin!!',
 '8,0,Sentiment140,       Sunny Again        Work Tomorrow  :-|       TV Tonight',
 '9,1,Sentiment140,      handed in my uniform today . i miss you already']

### 2. Removendo o cabeçalho...

In [19]:
header = file.take(1)[0]
files = file.filter(lambda x : (x != header))
files.take(10)

['1,0,Sentiment140,                     is so sad for my APL friend.............',
 '2,0,Sentiment140,                   I missed the New Moon trailer...',
 '3,1,Sentiment140,              omg its already 7:30 :O',
 "4,0,Sentiment140,          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...",
 '5,0,Sentiment140,         i think mi bf is cheating on me!!!       T_T',
 '6,0,Sentiment140,         or i just worry too much?        ',
 '7,1,Sentiment140,       Juuuuuuuuuuuuuuuuussssst Chillin!!',
 '8,0,Sentiment140,       Sunny Again        Work Tomorrow  :-|       TV Tonight',
 '9,1,Sentiment140,      handed in my uniform today . i miss you already',
 '10,1,Sentiment140,      hmmmm.... i wonder how she my number @-)']